In [1]:
import pandas as pd
import csv
import requests
from pprint import pprint
import json
import numpy as np
from citipy import citipy

## Yelp Business Data

In [2]:
y_bus = pd.read_csv("yelp_business.csv")

In [3]:
y_bus.head()

,business_id,categories,latitude,review_count,city,stars
0,vcNAWiLM4dR7D2nwwJ7nCA,Doctors;Health & Medical,33.499313,7,Phoenix,3.5
1,JwUE5GmEO-sH1FuwJgKBlQ,Restaurants,43.238893,26,De Forest,4.0
2,uGykseHzyS5xAMWoN6YUqA,American (Traditional);Restaurants,43.252267,16,De Forest,4.0
3,LRKJF43s9-3jG9Lgx4zODg,Food;Ice Cream & Frozen Yogurt;Fast Food;Resta...,43.251045,7,De Forest,4.5
4,RgDg-k9S5YD_BaxMckifkg,Chinese;Restaurants,43.240875,3,De Forest,4.0


In [4]:
print(len(y_bus))

42153


In [ ]:
yelp_business = pd.DataFrame(y_bus)
yelp_business.head()

In [ ]:
type(yelp_business)

In [ ]:
yelp_business.isna().sum()

In [ ]:
yelp_business = yelp_business.dropna(how='any')

In [ ]:
yelp_business.isna().sum()

In [ ]:
yelp_business.duplicated().sum()

In [ ]:
yelp_business.describe()

### Yelp Reviews

In [ ]:
y_rev = pd.read_csv("yelp_review.csv")

In [ ]:
y_rev.head()

In [ ]:
print(len(y_rev))

In [ ]:
yelp_reviews = pd.DataFrame(y_rev)
yelp_reviews.head()

In [ ]:
type(yelp_reviews)

In [ ]:
yelp_reviews.isna().sum()

In [ ]:
yelp_reviews = yelp_reviews.dropna(how = 'any')

In [ ]:
yelp_reviews.isna().sum()

In [ ]:
yelp_reviews.duplicated().sum()

In [ ]:
yelp_reviews = yelp_reviews.drop_duplicates()

In [ ]:
yelp_reviews.duplicated().sum()

## Zomato API

In [5]:
url = "https://developers.zomato.com/api/v2.1/search?"

In [6]:
header = {"Accept": "application/json", "user_key": "0092256f05a628b53994aec9ae1f30e0"}

response = requests.get(url, headers=header).json()

print(url)

https://developers.zomato.com/api/v2.1/search?


In [8]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
total = len(cities)
total

635

In [ ]:
%pprint
pprint(response)

In [16]:
query_url = "https://developers.zomato.com/api/v2.1/cities?q="
city_num = 0
api_data = open('output_data/locations.csv', 'w')
csvwriter = csv.writer(api_data)
header = ['Location ID']
csvwriter.writerow(header)

for city in cities:
    url_header = {"Accept": "application/json", "user_key": "4ae5aec1f3577713b3ece44fc9c3d123"}
    response = requests.get(query_url + city, headers=url_header).json()
    city_num += 1
    #try:
    csvwriter.writerow([response['location_suggestions']['id']])
    print(f"Processing Record {city_num} of {total} | {city}")
    #except:
        #print(f"City not found. Skipping...")
        #continue
api_data.close()

TypeError: list indices must be integers or slices, not str

In [17]:
print(response)

{'location_suggestions': [{'id': 3659, 'name': 'Thompson, MB', 'country_id': 37, 'country_name': 'Canada', 'country_flag_url': 'https://b.zmtcdn.com/images/countries/flags/country_37.png', 'should_experiment_with': 0, 'discovery_enabled': 0, 'has_new_ad_format': 0, 'is_state': 0, 'state_id': 122, 'state_name': 'Manitoba', 'state_code': 'MB'}, {'id': 3075, 'name': 'Thompson-Nicola, BC', 'country_id': 37, 'country_name': 'Canada', 'country_flag_url': 'https://b.zmtcdn.com/images/countries/flags/country_37.png', 'should_experiment_with': 0, 'discovery_enabled': 0, 'has_new_ad_format': 0, 'is_state': 0, 'state_id': 121, 'state_name': 'British Columbia', 'state_code': 'BC'}, {'id': 3082, 'name': 'Thompson-Nicola A, BC', 'country_id': 37, 'country_name': 'Canada', 'country_flag_url': 'https://b.zmtcdn.com/images/countries/flags/country_37.png', 'should_experiment_with': 0, 'discovery_enabled': 0, 'has_new_ad_format': 0, 'is_state': 0, 'state_id': 121, 'state_name': 'British Columbia', 'state

In [ ]:
query_url = "https://developers.zomato.com/api/v2.1/search?entity_id="
city_num = 0
api_data = open('output_data/zomato.csv', 'w')
csvwriter = csv.writer(api_data)
header = ['City', 'Restaurant', 'Lat', 'Lng', 'Rating', 'Rating Text']
csvwriter.writerow(header)

for city in cities:
    url_header = {"Accept": "application/json", "user_key": "0092256f05a628b53994aec9ae1f30e0"}
    response = requests.get(query_url + city, headers=url_header).json()
    city_num += 1
    try:
        csvwriter.writerow([response['location']['city'], response['name'], response['location']['latitude'],
                    response['location']['longitude'], response['user_rating']['aggregate_rating'], response['user_rating']['rating_text']])
        print(f"Processing Record {city_num} of {total} | {city}")
    except:
        print(f"City not found. Skipping...")
        continue
api_data.close()

In [ ]:
url -X GET --header "Accept: application/json" --header "user-key: 4ae5aec1f3577713b3ece44fc9c3d123 " "https://developers.zomato.com/api/v2.1/categories"
